#### Basic package & directory settings
- network_wrangler branch: [bicounty_ml_connectors](https://github.com/wsp-sag/network_wrangler/tree/bicounty_ml_connectors)
- Lasso branch: [bicounty_emme](https://github.com/wsp-sag/Lasso/tree/bicounty_emme)

In [1]:
import os
import sys
import yaml
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger
from network_wrangler.transitnetwork import DotDict

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc
from lasso import util
from lasso import bicounty

import logging
logger = logging.getLogger("WranglerLogger")
logger.setLevel(logging.INFO)

%load_ext autoreload
%autoreload 2

In [2]:
data_dir = "../../data/"
lasso_dir = "../../software/Lasso/"

parameters = Parameters(lasso_base_dir = lasso_dir)

2024-03-22 18:28:16, INFO: Lasso base directory set as: ../../software/Lasso/
2024-03-22 18:28:16, INFO: Lasso base directory set as: ../../software/Lasso/


##### Load current scenario pickle file

In [3]:
pickle_file_name = os.path.join(data_dir, "scenario_pickle", "scenario_2020.pickle")
curr_scenario = pickle.load(open(pickle_file_name, 'rb'))

#### Create subarea highway network for Alameda county

In [4]:
temp_links = curr_scenario.road_net.links_df.copy()
temp_nodes = curr_scenario.road_net.nodes_df.copy()
temp_shapes = curr_scenario.road_net.shapes_df.copy()


# filter out links, node, shapes for subarea
subarea_links = temp_links[temp_links["county"] == "Alameda"].copy()

subarea_nodes_list = list(set(subarea_links["A"].unique().tolist() + subarea_links["B"].unique().tolist()))
subarea_nodes = temp_nodes[temp_nodes["model_node_id"].isin(subarea_nodes_list)].copy()

subarea_shapes = temp_shapes[temp_shapes["id"].isin(subarea_links["id"].unique().tolist())].copy()

In [5]:
# create subarea roadway network
subarea_road_net = RoadwayNetwork(links = subarea_links,
                                  nodes = subarea_nodes,
                                  shapes = subarea_shapes,
                                  # carry over other roadway network attributes from curr_scenario
                                  crs = curr_scenario.road_net.crs,
                                  node_foreign_key = curr_scenario.road_net.node_foreign_key,
                                  link_foreign_key = curr_scenario.road_net.link_foreign_key,
                                  shape_foreign_key = curr_scenario.road_net.shape_foreign_key,
                                  unique_link_ids = curr_scenario.road_net.unique_link_ids,
                                  unique_node_ids = curr_scenario.road_net.unique_node_ids,
                                  modes_to_network_link_variables = curr_scenario.road_net.modes_to_network_link_variables,
                                  modes_to_network_nodes_variables = curr_scenario.road_net.modes_to_network_nodes_variables)

#### Create subarea transit network for Alameda county

In [6]:
trn_agency = curr_scenario.transit_net.feed.agency.copy()
trn_routes = curr_scenario.transit_net.feed.routes.copy()
trn_trips = curr_scenario.transit_net.feed.trips.copy()
trn_stops = curr_scenario.transit_net.feed.stops.copy()
trn_frequencies = curr_scenario.transit_net.feed.frequencies.copy()
trn_shapes = curr_scenario.transit_net.feed.shapes.copy()
trn_stop_times = curr_scenario.transit_net.feed.stop_times.copy()
trn_fare_attributes = curr_scenario.transit_net.feed.fare_attributes.copy()
trn_fare_rules = curr_scenario.transit_net.feed.fare_rules.copy()

In [7]:
def get_subarea_trn_shapes(trn_shapes, subarea_nodes_list=subarea_nodes_list) -> list:

    # get shape_id that has all the nodes within subset area
    subarea_nodes_list_str = [str(x) for x in subarea_nodes_list] # in transit_net, shape_model_node_id is currently in str format 

    # helper function
    def is_within_subarea(row, subarea_nodes_list_str=subarea_nodes_list_str):
        return set(row["shape_model_node_id"]).issubset(subarea_nodes_list_str)

    # group trn_shapes by shape_id, and gather "shape_model_node_id" into list format
    trn_shape_nodes = trn_shapes.groupby("shape_id")["shape_model_node_id"].apply(list).reset_index()
    # for each row, check if the list in "shape_model_node_id" is subset of subarea_nodes_list_str (means all these nodes are within subarea)
    trn_shape_nodes["within_subarea"] = trn_shape_nodes.apply(is_within_subarea, axis=1)

    # returns trn_shapes that are inside subarea 
    trn_shape_nodes = trn_shape_nodes[trn_shape_nodes["within_subarea"] == True]
    subarea_trn_shapes = trn_shapes[trn_shapes["shape_id"].isin(trn_shape_nodes["shape_id"].tolist())].copy()

    return subarea_trn_shapes


def get_subarea_trn_feed(trn_shapes, subarea_nodes_list=subarea_nodes_list):
    
    # get subarea_trn_shapes first, then use it to process other transit info
    subarea_trn_shapes = get_subarea_trn_shapes(trn_shapes)

    subarea_trn_trips = trn_trips[trn_trips["shape_id"].isin(subarea_trn_shapes["shape_id"].tolist())].copy()
    subarea_trn_stop_times = trn_stop_times[trn_stop_times["trip_id"].isin(subarea_trn_trips["trip_id"].tolist())].copy()
    subarea_trn_stops = trn_stops[trn_stops["stop_id"].isin(subarea_trn_stop_times["stop_id"].tolist())].copy()
    subarea_trn_routes = trn_routes[trn_routes["route_id"].isin(subarea_trn_trips["route_id"].tolist())].copy()
    subarea_trn_agency = trn_agency[trn_agency["agency_id"].isin(subarea_trn_routes["agency_id"].tolist())].copy()
    subarea_trn_frequencies = trn_frequencies[trn_frequencies["trip_id"].isin(subarea_trn_trips["trip_id"].tolist())].copy()
    subarea_trn_fare_rules = trn_fare_rules[trn_fare_rules["agency_raw_name"].isin(subarea_trn_agency["agency_raw_name"].tolist())].copy()
    subarea_trn_fare_attributes = trn_fare_attributes[trn_fare_attributes["fare_id"].isin(subarea_trn_fare_rules["fare_id"].tolist())].copy()

    # create transit feed
    subarea_trn_feed = DotDict()
    subarea_trn_feed["agency"] = subarea_trn_agency
    subarea_trn_feed["routes"] = subarea_trn_routes
    subarea_trn_feed["trips"] = subarea_trn_trips
    subarea_trn_feed["stops"] = subarea_trn_stops
    subarea_trn_feed["frequencies"] = subarea_trn_frequencies
    subarea_trn_feed["shapes"] = subarea_trn_shapes
    subarea_trn_feed["stop_times"] = subarea_trn_stop_times
    subarea_trn_feed["fare_attributes"] = subarea_trn_fare_attributes
    subarea_trn_feed["fare_rules"] = subarea_trn_fare_rules

    return subarea_trn_feed

In [8]:
# create transit network for subarea
subarea_trn_feed = get_subarea_trn_feed(trn_shapes, subarea_nodes_list)
subarea_transit_net = TransitNetwork(feed = subarea_trn_feed,
                                     # carry over other attributes from current transit_net
                                     config = curr_scenario.transit_net.config,
                                     shapes_foreign_key = curr_scenario.transit_net.shapes_foreign_key,
                                     stops_foreign_key = curr_scenario.transit_net.stops_foreign_key,
                                     id_scalar = curr_scenario.transit_net.id_scalar)

#### Create scenario from subarea network

In [9]:
subarea_scenario = Scenario.create_scenario(base_scenario= {"road_net": subarea_road_net, "transit_net": subarea_transit_net})

2024-03-22 18:36:34, INFO: Creating Scenario


#### Apply example project cards to subset scenario

In [10]:
# apply highway project cards
hwy_card_dir = "../../data/project_cards_example/test_alameda/hwy/"
hwy_project_card_list = [# highway project cards
                         "test_alameda_ml.yml",
                         "test_alameda_lane_widening.yml",
                         "test_alameda_grade_separation.yml",
                         # highway project card for transit
                         "test_alameda_add_rail_links.yml"]

for hwy_project in hwy_project_card_list:
    _filename = os.path.join(hwy_card_dir, hwy_project)
    card = ProjectCard.read(_filename, validate = False)
    subarea_scenario.apply_project(card)

2024-03-22 18:36:35, INFO: Applying [../../data/project_cards_example/test_alameda/hwy/test_alameda_ml.yml]:Example - convert 1 GP lane to Express Lane
2024-03-22 18:36:35, INFO: Applying [../../data/project_cards_example/test_alameda/hwy/test_alameda_ml.yml]:Example - convert 1 GP lane to Express Lane
2024-03-22 18:36:35, INFO: Applying Project to Roadway Network: Example - convert 1 GP lane to Express Lane
2024-03-22 18:36:35, INFO: Applying [../../data/project_cards_example/test_alameda/hwy/test_alameda_lane_widening.yml]:Example - Number of lane changes
2024-03-22 18:36:35, INFO: Applying [../../data/project_cards_example/test_alameda/hwy/test_alameda_lane_widening.yml]:Example - Number of lane changes
2024-03-22 18:36:35, INFO: Applying Project to Roadway Network: Example - Number of lane changes
2024-03-22 18:36:35, INFO: Applying [../../data/project_cards_example/test_alameda/hwy/test_alameda_grade_separation.yml]:Example - Grade Separation
2024-03-22 18:36:35, INFO: Applying [.

c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2024-03-22 18:36:35, INFO: Applying Project to Transit Network: Example - Grade Separation
2024-03-22 18:36:35, WARNING: Roadway Deletion not yet implemented in Transit; ignoring
2024-03-22 18:36:35, INFO: Applying [../../data/project_cards_example/test_alameda/hwy/test_alameda_grade_separation.yml]:Example - Grade Separation
2024-03-22 18:36:35, INFO: Applying Project to Roadway Network: Example - Grade Separation


c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


2024-03-22 18:37:05, INFO: Applying [../../data/project_cards_example/test_alameda/hwy/test_alameda_add_rail_links.yml]:Example - Extend BART Oakland Airport Connector - add rail link
2024-03-22 18:37:05, INFO: Applying [../../data/project_cards_example/test_alameda/hwy/test_alameda_add_rail_links.yml]:Example - Extend BART Oakland Airport Connector - add rail link
2024-03-22 18:37:05, INFO: Applying Project to Roadway Network: Example - Extend BART Oakland Airport Connector - add rail link


c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [11]:
# apply transit project cards
subarea_scenario.road_net.links_df = gpd.GeoDataFrame(subarea_scenario.road_net.links_df, geometry=subarea_scenario.road_net.links_df.geometry)
subarea_scenario.road_net.nodes_df = gpd.GeoDataFrame(subarea_scenario.road_net.nodes_df, geometry=subarea_scenario.road_net.nodes_df.geometry)
subarea_scenario.transit_net.set_roadnet(subarea_scenario.road_net, validate_consistency=False)


trn_card_dir = "../../data/project_cards_example/test_alameda/trn/"
trn_project_card_list = ["test_alameda_bart_extension.wrangler"]

for trn_project in trn_project_card_list:
    _filename = os.path.join(trn_card_dir, trn_project)
    card = ProjectCard.read(_filename, validate = False)
    subarea_scenario.apply_project(card)

2024-03-22 18:38:02, INFO: Applying [../../data/project_cards_example/test_alameda/trn/test_alameda_bart_extension.wrangler]:Example - Extend BART Oakland Airport Connector
2024-03-22 18:38:02, INFO: Applying Project to Transit Network: Example - Extend BART Oakland Airport Connector


#### Check if highway project cards got applied

In [12]:
check_links = subarea_scenario.road_net.links_df.copy()

In [13]:
# check example ML
check_ml = check_links[check_links["model_link_id"].isin([3017447, 3040508])]
# should see changes on managed-lane related attributes
check_ml[["A", "B", "model_link_id", "lanes", "managed", "ML_lanes", "ML_useclass", "ML_access", "ML_egress"]]

,A,B,model_link_id,lanes,managed,ML_lanes,ML_useclass,ML_access,ML_egress
17447,2517104,2537598,3017447,"{'default': 4.0, 'timeofday': [{'time': (21600...",1,"{'default': 0, 'timeofday': [{'time': (21600, ...","{'default': 0, 'timeofday': [{'time': (21600, ...",all,all
40508,2537598,2593478,3040508,"{'default': 4.0, 'timeofday': [{'time': (21600...",1,"{'default': 0, 'timeofday': [{'time': (21600, ...","{'default': 0, 'timeofday': [{'time': (21600, ...",all,all


In [14]:
# check example lane widening
check_lanes = check_links[check_links["model_link_id"].isin([3102795, 3059282])]
# should see 3 lanes
check_lanes[["A", "B", "model_link_id", "lanes"]]

,A,B,model_link_id,lanes
59281,2552579,2605114,3059282,3
102791,2516137,2552579,3102795,3


In [15]:
# check example grade separation - (1) remove links
check_rm_links = check_links[check_links["model_link_id"].isin([3063899, 3233901, 3059616, 3297254])]

# this should be empty
check_rm_links

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg,cntype


In [16]:
# check example grade separation - (2) add new links
check_new_links = check_links[((check_links["A"] == 2590965) & (check_links["B"] == 2625980)) | 
                              ((check_links["A"] == 2625980) & (check_links["B"] == 2590965)) |
                              ((check_links["A"] == 2625980) & (check_links["B"] == 2553367)) |
                              ((check_links["A"] == 2553367) & (check_links["B"] == 2625980))]
check_new_links[["A", "B", "model_link_id", "drive_access", "bike_access", "walk_access", "ft", "lanes"]]

,A,B,model_link_id,drive_access,bike_access,walk_access,ft,lanes
328148,2590965,2625980,26067371,1.0,1.0,1.0,4.0,2
328149,2625980,2590965,26067372,1.0,1.0,1.0,4.0,2
328150,2625980,2553367,26067373,1.0,1.0,1.0,4.0,2
328151,2553367,2625980,26067374,1.0,1.0,1.0,4.0,2


#### Save subset scenario to pickle file

In [17]:
subset_scenario_filename = os.path.join(data_dir, "scenario_pickle", "alameda_scenario_2020.pickle")
pickle.dump(subarea_scenario, open(subset_scenario_filename, 'wb'))

#### Convert to model network format

In [18]:
model_roadway = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = subarea_scenario.road_net, 
    parameters = parameters
)

2024-03-22 18:38:45, INFO: Lasso base directory set as: ../../software/Lasso/
2024-03-22 18:38:45, INFO: Lasso base directory set as: ../../software/Lasso/
2024-03-22 18:38:45, INFO: Filling nan for network from network wrangler
2024-03-22 18:38:52, INFO: Converting variable type to mtc standard


In [19]:
model_network = mtc.roadway_standard_to_mtc_network(model_roadway, parameters=parameters)

2024-03-22 18:38:53, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2024-03-22 18:38:53, INFO: Creating managed lane network.
2024-03-22 18:38:53, INFO: Creating network with duplicated managed lanes


c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\geopandas\geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert

2024-03-22 18:42:10, INFO: Determining cntype
2024-03-22 18:42:10, INFO: Lasso base directory set as: ../../software/Lasso/
2024-03-22 18:42:10, INFO: Lasso base directory set as: ../../software/Lasso/
2024-03-22 18:42:10, INFO: Overwriting existing Variable 'cntype' already in network
2024-03-22 18:42:10, INFO: Calculating and adding roadway network variable: cntype
2024-03-22 18:42:21, INFO: Finished determining variable: cntype
2024-03-22 18:42:21, INFO: Determining transit
2024-03-22 18:42:21, INFO: Lasso base directory set as: ../../software/Lasso/
2024-03-22 18:42:21, INFO: Lasso base directory set as: ../../software/Lasso/
2024-03-22 18:42:21, INFO: Variable 'transit' updated for some links. Returning without overwriting for those links. Calculating for other links
2024-03-22 18:42:21, INFO: Calculating and adding roadway network variable: transit
2024-03-22 18:42:21, INFO: Finished determining transit-only variable: transit
2024-03-22 18:42:21, INFO: Determining useclass
2024-0

d:\bcm\software\lasso\lasso\mtc.py:3112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  link_centroids_gdf["geometry"] = link_centroids_gdf["geometry"].centroid
c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  """Entry point for launching an IPython kernel.


2024-03-22 18:43:15, INFO: Finished Calculating link county variable: county


c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  """Entry point for launching an IPython kernel.


2024-03-22 18:43:24, INFO: Finished Calculating node county variable: county
2024-03-22 18:43:25, INFO: Overwriting existing distance Variable 'distance' already in network
2024-03-22 18:43:35, INFO: Calculating distance for all links
2024-03-22 18:43:36, INFO: Calculating distance for all links
2024-03-22 18:43:37, INFO: Filling nan for network from network wrangler
2024-03-22 18:43:42, INFO: Splitting variables by time period and category
2024-03-22 18:43:43, WARNING: Specified variable to split: ML_lanes not in network variables: Index(['index', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'id', 'nodeIds', 'roadClass', 'roundabout',
       'link', 'highway', 'oneway', 'service', 'bridge', 'junction', 'tunnel',
  

In [20]:
model_network.links_mtc_df['name'] = model_network.links_mtc_df['name'].apply(lambda x: "" if type(x) == int else x)
model_network.links_mtc_df['name'] = model_network.links_mtc_df['name'].apply(lambda x: util.shorten_name(x))

##### Write out to Cube network

In [21]:
# write cube highway network
model_network.write_roadway_as_fixedwidth(
    output_dir = "../../data/cube_network/alameda_example/hwy",
    output_link_txt = 'links.txt',
    output_node_txt = 'nodes.txt',
    output_link_header_width_txt = 'links_header_width.txt',
    output_node_header_width_txt = 'nodes_header_width.txt',
    output_cube_network_script = 'make_complete_network_from_fixed_width_file.s',
)

2024-03-22 18:44:08, INFO: Starting fixed width conversion


c:\Users\USYH688633\.conda\envs\bicounty\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


2024-03-22 18:46:12, INFO: Writing out link database
2024-03-22 18:46:15, INFO: Writing out link header and width ----
2024-03-22 18:46:15, INFO: Starting fixed width conversion
2024-03-22 18:46:32, INFO: Writing out node database
2024-03-22 18:46:32, INFO: Writing out node header and width
2024-03-22 18:46:32, INFO: Running ["C:\Program Files\Citilabs\CubeVoyager\runtpp.exe" make_complete_network_from_fixed_width_file.s] in cwd [../../data/cube_network/alameda_example/hwy]
2024-03-22 18:46:45, INFO: return code: 0
2024-03-22 18:46:45, INFO: stdout: PILOT    (v.07/10/2023 [6.5.1 x64]) Fri Mar 22 18:46:34 2024
2024-03-22 18:46:45, INFO: stdout: Print:   d:\bcm\data\cube_network\alameda_example\hwy\TPPL0002.PRN
2024-03-22 18:46:45, INFO: stdout: Input:   d:...da_example\hwy\make_complete_network_from_fixed_width_file.s
2024-03-22 18:46:45, INFO: stdout: NETWORK  (v.07/10/2023 [6.5.1 x64]) Fri Mar 22 18:46:34 2024
                                                                           

In [22]:
# write cube transit network
# note: there needs to be "faresystem_crosswalk.txt" file under the same directory where transit.lin is writing out to
standard_transit_net = StandardTransit.fromTransitNetwork(subarea_scenario.transit_net, parameters=parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = "../../data/cube_network/alameda_example/trn/alameda_transit.lin")

2024-03-22 18:46:46, INFO: Lasso base directory set as: ../../software/Lasso/
2024-03-22 18:46:46, INFO: Lasso base directory set as: ../../software/Lasso/
2024-03-22 18:46:46, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
